<a href="https://colab.research.google.com/github/sagar-demo/Deep-Learning/blob/master/Fake_News_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
###Fake News Classifier Using LSTM
##Datasets:https://wwww.kaggle.com/c/Fake-news/data

In [2]:
import pandas as pd
df=pd.read_csv('/content/drive/My Drive/Deep_learning/data/train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
## Drop na value
df=df.dropna()

In [4]:
### GET the independent features
X=df.drop('label',axis=1)


In [5]:
###Get the dependent varibale/features
y=df['label']

In [6]:
X.shape

(18285, 4)

In [7]:
y.shape

(18285,)

In [8]:
import tensorflow as tf

In [9]:
tf.__version__

'2.3.0'

In [10]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [11]:
###Vocabulary size
voc_size=5000

In [12]:
###ONE hot reprensentation
message=X.copy()

In [13]:
message['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [14]:
message.reset_index(inplace=True)

In [15]:
import nltk
import re
from nltk.corpus import stopwords

In [16]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
#### Data preprocessing
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]
for i in range(0,len(message)):
  print(i)
  review=re.sub('[^a-zA-Z]',' ',message['title'][i])
  review=review.lower()
  review=review.split()

  review=[ps.stem(word) for  word in review if not word in stopwords.words('english')]
  review=' '.join(review)
  corpus.append(review)




Streaming output truncated to the last 5000 lines.
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
13443
1

In [18]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [19]:
one_hot_rep=[one_hot(word,voc_size)for word in corpus]
one_hot_rep

[[1724, 3509, 3766, 4183, 2925, 2052, 4045, 4072, 2911, 2415],
 [1240, 3073, 1126, 2862, 4632, 3473, 717],
 [4081, 639, 4418, 1063],
 [3130, 313, 1256, 3946, 470, 1961],
 [4229, 4632, 4008, 1737, 1204, 3923, 4632, 3666, 197, 2513],
 [4251,
  4383,
  297,
  10,
  4271,
  3929,
  2456,
  1515,
  3942,
  1251,
  1502,
  2536,
  897,
  4736,
  717],
 [298, 1717, 1336, 1077, 917, 3662, 1202, 227, 1774, 4778, 1882],
 [4259, 2345, 2614, 2806, 2590, 2297, 3929, 3617, 1774, 4778, 1882],
 [2091, 2967, 3582, 1261, 2032, 4211, 377, 111, 3929, 850],
 [3341, 1592, 4273, 673, 385, 3940, 912, 162],
 [3144, 4867, 3357, 3991, 598, 2978, 2186, 597, 1707, 467, 1784],
 [3946, 1390, 2925, 4211, 3929, 2590],
 [3582, 2462, 1596, 3224, 2098, 2928, 4972, 3241, 3855],
 [591, 1966, 3275, 1104, 2048, 1962, 2483, 1774, 4778, 1882],
 [3269, 665, 1133, 2511, 2380, 1774, 4778, 1882],
 [559, 894, 66, 4003, 3016, 1123, 2214, 1117, 4117, 3801],
 [2850, 3343, 3073],
 [1944, 3669, 1092, 2841, 3929, 1797, 3537, 717],
 [3836

In [20]:
###Embedding Representation
sent_length=20
embedded_docs=pad_sequences(one_hot_rep,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4072 2911 2415]
 [   0    0    0 ... 4632 3473  717]
 [   0    0    0 ...  639 4418 1063]
 ...
 [   0    0    0 ... 1774 4778 1882]
 [   0    0    0 ... 2274 2893 3707]
 [   0    0    0 ... 1926 2318 3556]]


In [21]:
len(embedded_docs)

18285

In [22]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1724,
       3509, 3766, 4183, 2925, 2052, 4045, 4072, 2911, 2415], dtype=int32)

In [23]:
### creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
len(embedded_docs),y.shape

(18285, (18285,))

In [25]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
X_train,X_test,y_train,y_test=train_test_split(X_final,y_final,test_size=0.33,random_state=42)

In [28]:
###model training

In [33]:
#final training 
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 6s 33ms/step - loss: 0.3445 - accuracy: 0.8357 - val_loss: 0.2160 - val_accuracy: 0.9104
Epoch 2/10
192/192 [==============================] - 6s 33ms/step - loss: 0.1507 - accuracy: 0.9408 - val_loss: 0.1948 - val_accuracy: 0.9181
Epoch 3/10
192/192 [==============================] - 6s 31ms/step - loss: 0.1154 - accuracy: 0.9594 - val_loss: 0.2260 - val_accuracy: 0.9168
Epoch 4/10
192/192 [==============================] - 6s 31ms/step - loss: 0.0891 - accuracy: 0.9700 - val_loss: 0.2319 - val_accuracy: 0.9140
Epoch 5/10
192/192 [==============================] - 6s 31ms/step - loss: 0.0695 - accuracy: 0.9767 - val_loss: 0.2813 - val_accuracy: 0.9155
Epoch 6/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0572 - accuracy: 0.9804 - val_loss: 0.3123 - val_accuracy: 0.9133
Epoch 7/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0439 - accuracy: 0.9850 - val_loss: 0.3098 - val_accuracy: 0.9133

In [30]:
###Adding Dropout(hyperparameter tunning)
from tensorflow.keras.layers import Dropout
### creating model
embedding_vector_feature=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_feature,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [34]:
### Performance Metrics And Accuracy
y_pred=model.predict_classes(X_test)
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test,y_pred)

array([[3082,  337],
       [ 205, 2411]])

In [35]:
accuracy_score(y_test,y_pred)

0.9101905550952776